In [ ]:
import numpy as np
import torch
from torch import nn
import torch.optim as optim
import os
import math
from tqdm import tqdm
from tqdm import trange
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import random


In [ ]:
class PathsDataset(torch.utils.data.Dataset):
    # init the dataset, shape = L x W
    def __init__(self, path_dir, map_file, transform=None, shape = (100,100), device='cpu'):
        self.device = device
        self.paths = [] # create a list to hold all paths read from file
        # self.map = np.loadtxt(map_file, skiprows=2).reshape(shape)
        # self.map = self.map[np.newaxis, :, :]
        x = torch.tensor([]) # empty list to hold input series tensors
        num_paths = 0
        for filename in tqdm(os.listdir(path_dir)):
            num_paths += 1
            with open(os.path.join(path_dir, filename), 'r') as f: # open in readonly mode
                path_points_list = [] # a list to hold each point in a path
                self.flat_path = np.loadtxt(f) # load in the flat path from file
                self.path = np.asarray(self.flat_path, dtype=np.float32).reshape(len(self.flat_path)//2,2) #unflatten the path from the file
                # print(self.path)
                for point in self.path:
                    x = point[0]
                    y = point[1]
                    this_point = [x, y]
                    path_points_list.append(this_point)
            sequence = torch.tensor(path_points_list, dtype=torch.float)[:, :]
            self.paths.append(sequence)
                # self.path_tensor = self.convert_path(shape, self.path)

        # self.sequences = []
        # for path in range(len(path_list)):
        #    self.sequences.append((path_list[path] - mu)/sig)
        
        self.transform = transform
        print("Done!")

    def convert_path(self, map_dim, path):
        path_mat = np.zeros(map_dim, dtype=float)

        # Make the path continuous
        for i in range(path.shape[0] - 1):
            x = path[i,0]
            x1 = path[i,0]
            x2 = path[i+1,0]

            y = path[i,1]
            y1 = path[i,1]
            y2 = path[i+1,1]

            if (x1 < x2):
                x_dir = 1
            else:
                x_dir = -1

            if (y1 < y2):
                y_dir = 1
            else:
                y_dir = -1

            # Determine y from x
            if x2-x1 != 0:
                m = (y2-y1)/(x2-x1)
                while x != x2:
                    y = round(m*(x-x1) + y1)
                    path_mat[y,x] = 1
                    x += x_dir
            else:
                while x != x2:
                    path_mat[y1,x] = 1
                    x += x_dir


            x = path[i,0]
            x1 = path[i,0]
            x2 = path[i+1,0]

            y = path[i,1]
            y1 = path[i,1]
            y2 = path[i+1,1]

            # Determine x from y
            if y2-y1 != 0:
                m = (x2-x1)/(y2-y1)
                while y != y2:
                    x = round(m*(y-y1) + x1)
                    path_mat[y,x] = 1
                    y += y_dir
            else:
                while y != y2:
                    path_mat[y,x1] = 1
                    y += y_dir

        # print(path)
        # print(f'xs: {path[:,0]}')
        # print(f'ys: {path[:,1]}')
        # path_tensor = torch.tensor(path)
        # print(path_tensor)
        return path

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        # x = np.float32(self.sequences[idx])
        # x = torch.Tensor(x).to(self.device)

        x = self.paths[idx]
        if self.transform:
            x = self.transform(x)
        
        return x

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# dataset name
MAP_NAME = '8x12_map_cropped'
DATASET = 'random_paths'
MAP_SHAPE = (64,64)
# training parameters
BATCH_SIZE = 1
train_dataset = PathsDataset(path_dir = f"./env/{MAP_NAME}/paths/{DATASET}/dense", map_file = f"./env/{MAP_NAME}/{MAP_NAME}.txt", shape = MAP_SHAPE, transform=None, device=device)
dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)

In [22]:
class Encoder(nn.Module):
    
    def __init__(self):
        super().__init__()
        
        ### Convolutional section
        self.encoder_cnn = nn.Sequential(
            nn.Conv2d(1, 16, 5, stride=2, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(True),
            nn.Conv2d(16, 32, 5, stride=2, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(True),
            nn.Conv2d(32, 64, 5, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(True),     
            nn.Conv2d(64, 128, 7, stride=2, padding=0),
            nn.BatchNorm2d(128),
            nn.ReLU(True),  
        )
        
        ### Flatten layer
        self.flatten = nn.Flatten(start_dim=1)
        
    def forward(self, x):
        x = self.encoder_cnn(x)
        x = self.flatten(x)
        return x

In [23]:
# create a new encoder
encoder = Encoder()

In [ ]:
#todo: load encoder checkpoint from file


In [ ]:
#todo: run the map through the encoder and get an encoded map to use in the lstm

In [ ]:
train_target_paths = []
train_input_paths = []

test_target_paths = []
test_input_paths = []

nbr_eval_paths = 10

for path in train_dataset[nbr_eval_paths:]:
    input_path = path[:-1] #all points execpt last
    target_path = path[1:]

    train_target_paths.append(target_path)
    train_input_paths.append(input_path)

for path in train_dataset[:nbr_eval_paths]:
    input_path = path[:-1] #all points execpt last five
    target_path = path[1:]

    test_target_paths.append(target_path)
    test_input_paths.append(input_path)


print(len(train_input_paths))
print(len(test_input_paths))
train_input_paths[3]

In [ ]:
class MyLSTM(nn.Module):
  # hidden_d - the size of the hidden LSTM layers
  # map_d - the flattened/encoded map dimension
  def __init__(self, hidden_d=120, map_d=100):
    self.hidden_d = hidden_d
    super(MyLSTM, self).__init__()

    # map hidden layer
    self.lstm_map = nn.LSTMCell(input_size=map_d, hidden_size=hidden_d)

    # points hidden layer
    self.lstm_points = nn.LSTMCell(input_size=4, hidden_size=hidden_d)
    
    # "upper" hidden layer
    self.lstm1 = nn.LSTMCell(input_size=hidden_d*2, hidden_size=hidden_d)
    self.fc = nn.Linear(hidden_d, 2)


  def forward(self, goal_point, current_point, map, future=0):

    

    # Creation of cell state and hidden state for map hidden layer
    hidden_state_map = torch.zeros(1, self.hidden_d)
    cell_state_map = torch.zeros(1, self.hidden_d)

    # Creation of cell state and hidden state for points hidden layer
    hidden_state_points = torch.zeros(1, self.hidden_d)
    cell_state_points = torch.zeros(1, self.hidden_d)

    # Creation of cell state and hidden state for "upper" hidden layer
    hidden_state_1 = torch.zeros(1, self.hidden_d)
    cell_state_1 = torch.zeros(1, self.hidden_d)

    outputs = []


    # initialize weights to random[-0.1, 0.1) (need to update initialzation to match paper)
    # weights initialization
    torch.nn.init.xavier_normal_(hidden_state_map)
    torch.nn.init.xavier_normal_(cell_state_map)

    torch.nn.init.xavier_normal_(hidden_state_points)
    torch.nn.init.xavier_normal_(cell_state_points)

    torch.nn.init.xavier_normal_(hidden_state_1)
    torch.nn.init.xavier_normal_(cell_state_1)

    # Concatenate start and goal
    points = torch.cat([current_point, goal_point], 0)

    hidden_state_map, cell_state_map = self.lstm1(map, (hidden_state_map, cell_state_map))

    hidden_state_points, cell_state_points = self.lstm1(points, (hidden_state_points, cell_state_points))

    # Concatenate the output the lstm layer output from points and map into a single input to the final "upper" hidden layer
    final_layer_input = torch.cat([hidden_state_map, hidden_state_points], 0)
    hidden_state_1, cell_state_1 = self.lstm1(final_layer_input, (hidden_state_1, cell_state_1))
      
    # Last hidden state is passed through a fully connected neural net
    output = self.fc(hidden_state_1)	
    # print(f'out: {output}')
    outputs.append(output)

    for i  in range(future): # if we are trying to predict future values (if future is not zero)
      hidden_state_1, cell_state_1 = self.lstm1(x[i:i+1], (hidden_state_1, cell_state_1))
      
    # Last hidden state is passed through a fully connected neural net
      output = self.fc(hidden_state_1)	
      # print(f'out: {output}')
      outputs.append(output)

    outputs = torch.cat(outputs, dim=0)
    
    return outputs

In [ ]:
rnn = MyLSTM()
loss = []
criterion = nn.MSELoss()
opt = torch.optim.Adam(rnn.parameters(), lr=0.005)
# n_epochs = 1000
n_epochs = 1
for e in range(n_epochs):
  print(f'epoch: {e}')
  for s in range(len(train_input_paths)):
    # print(train_input_paths[s].shape)
    pred = rnn(train_input_paths[s])  # predict next step, init hidden state to zero at the begining of the sequence
    # print(f'pred: {pred.shape}')
    # print(f'target: {train_target_paths[s].shape}')
    err = criterion(pred, train_target_paths[s])  # predict next step for each step
    opt.zero_grad()
    err.backward()
    opt.step()
    loss.append(err.item())
    if s % 100 is 0:  
      print(err.item())
plt.plot(loss)
plt.ylabel('Loss')
plt.xlabel('iteration')
plt.show()

In [ ]:
flat_map = np.loadtxt("./env/8x12_map_cropped/8x12_map_cropped.txt", skiprows=2)
map = np.asarray(flat_map).reshape(64,64)

In [ ]:
# prediction loop
i = 3 # what path # from the eval dataset we want to look at
with torch.no_grad():
    future=15
    length = train_input_paths[i].shape[0]
    pred = rnn(train_input_paths[i], future=future)
    truth = train_target_paths[i]
    # loss = loss_func(pred[:, :-future], test_target)
    # print(f'loss: {loss.item()}')
    y = pred.detach().numpy()
print(length)
print(y.shape)

In [ ]:
# print(train_input_paths[3])
# print(start_point)

In [ ]:
xs = y[:,0]
ys = y[:,1]

plt.imshow(map)
plt.plot(xs[:length], ys[:length], color='k', label = "input")
plt.plot(xs[length:], ys[length:], color='b', label = "predicted")
plt.plot(truth[:,0], truth[:,1], color='r', label = "truth")
plt.legend()

In [ ]:
start_points_list = []
coords_list = []

num_points = 150 # num points to predict for each path
paths_to_gen = 6 # num paths to generate

for x in range(paths_to_gen):
    x = random.randint(0, 64)
    y = random.randint(0, 64)
    start_point = torch.from_numpy(np.asarray([[x,y]])).float()
    points = []
    with torch.no_grad():
        pred = rnn(start_point, future=0)
        points.append(pred)
        for point in range(num_points):
            pred = rnn(pred, future=0)
            points.append(pred)
        points = torch.cat(points, dim=0)

    coords_list.append(points)

In [ ]:



flat_map = np.loadtxt("./env/8x12_map_cropped/8x12_map_cropped.txt", skiprows=2)
map = np.asarray(flat_map).reshape(64,64)


fig, ax = plt.subplots(nrows=5, ncols=1, figsize=(25, 25))
for x in range(5):
    ax[x].scatter(coords_list[x][:,0], coords_list[x][:,1], label='predicted points')
    ax[x].scatter(coords_list[x][0,0], coords_list[x][0,1], label='start point')
    ax[x].legend()
    ax[x].imshow(map)
plt.show()
plt.tight_layout()